In [1]:
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D

In [2]:
def download_model(model_type):
    server_prefix = 'https://omnomnom.vision.rwth-aachen.de/data/metrabs'
    model_zippath = tf.keras.utils.get_file(
        origin=f'{server_prefix}/{model_type}_20211019.zip',
        extract=True, cache_subdir='models')
    model_path = os.path.join(os.path.dirname(model_zippath), model_type)
    print(model_path)
    return model_path

model = tf.saved_model.load(download_model('metrabs_eff2l_y4')) # or metrabs_mob3l_y4t for the small model

C:\Users\akushwaha2024\.keras\models\metrabs_eff2l_y4


In [3]:
def visualize(im, detections, poses3d, poses2d, edges):
    fig = plt.figure(figsize=(10, 5.2))
    image_ax = fig.add_subplot(1, 2, 1)
    image_ax.imshow(im)
    for x, y, w, h in detections[:, :4]:
        image_ax.add_patch(Rectangle((x, y), w, h, fill=False))

    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(5, -85)
    pose_ax.set_xlim3d(-1500, 1500)
    pose_ax.set_zlim3d(-1500, 1500)
    pose_ax.set_ylim3d(0, 3000)

    poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1]
    
    for pose3d, pose2d in zip(poses3d, poses2d):
        for i_start, i_end in edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
            pose_ax.plot(*zip(pose3d[i_start], pose3d[i_end]), marker='o', markersize=2)
        image_ax.scatter(*pose2d.T, s=2)
        pose_ax.scatter(*pose3d.T, s=2)

    fig.tight_layout()
    plt.show()

In [4]:
# Extract joint coordinates
def extract_joints_coordinates(poses3d, poses2d):
    for i, (pose3d, pose2d) in enumerate(zip(poses3d, poses2d)):
        print(f"Pose {i + 1}:")
        print("2D Joint Coordinates:")
        print(pose2d)
        print("3D Joint Coordinates:")
        print(pose3d)
        print("-" * 40)

In [5]:
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        image = tf.convert_to_tensor(frame, dtype=tf.uint8)
        pred = model.detect_poses(image, skeleton='smpl_24')
        
        extract_joints_coordinates(pred['poses3d'].numpy(), pred['poses2d'].numpy())
        visualize(
            frame, 
            pred['boxes'].numpy(),
            pred['poses3d'].numpy(),
            pred['poses2d'].numpy(),
            model.per_skeleton_joint_edges['smpl_24'].numpy()
        )
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# Run video processing
test_video_path = 'video1.mp4'  # Change to the actual video path
process_video(test_video_path)